In [8]:
import numpy as np
import pandas as pd

import stanza
import spacy
import nltk
import re
import json
import os
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rishi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
stanza.download('en')

2021-12-03 19:13:35 INFO: Downloading default packages for language: en (English)...
2021-12-03 19:13:36 INFO: File exists: C:\Users\rishi\stanza_resources\en\default.zip.
2021-12-03 19:13:42 INFO: Finished downloading models and saved to C:\Users\rishi\stanza_resources.


In [4]:
spacy_pipline = dict()
spacy_pipline['en'] = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [5]:
stemmer = dict()
stemmer['en'] = SnowballStemmer("english")

In [6]:
def preprocess_text(text, lang='en'):
    #remove html tags
    text = BeautifulSoup(text, "html.parser").get_text()

    #remove links
    text = re.sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "", text)

    #remove text between []
    text = re.sub('\[[^]]*\]', '', text)

    #fix contractions
    def decontracted(phrase):

        phrase = re.sub("\u2019", '\'', phrase)
        phrase = re.sub("\u2018", '\'', phrase)
        phrase = re.sub("\u201C", '\"', phrase)
        phrase = re.sub("\u201D", '\"', phrase)

        # specific
        phrase = re.sub(r"won\'t", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)

        return phrase

    text = decontracted(text)
    
    #Remove numbers
    text = re.sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "", text)

    #lowercase
    text = text.lower()

    # tokenize the string
    # remove punctuation and special characters   
    text = re.sub(r'[_]', ' ', text)
    text = re.sub(r'[\r|\n|\r\n]+', ' ', text) 
    tokenizer = RegexpTokenizer(r'\w+')
    
    text = tokenizer.tokenize(text)
    
    #Remove stopwords
    stopwords = set(get_stop_words(lang))
    text = [i for i in text if i not in stopwords]
    
    # #Stemming
    # text = [stemmer[lang].stem(word) for word in text]

    #Lemmatize
    text = [word for word in text if word is not None]
    doc = spacy_pipline[lang](' '.join(text))

    if lang == 'ar' or lang == 'tr':
        if (len(doc.sentences) != 0):
            text = [token.lemma for token in doc.sentences[0].words]
    else:
        text = [token.lemma_ for token in doc]

    return ' '.join(text)

In [17]:
root = ".\\fns2020_dataset\\training\\annual_reports"
dest = ".\\datasets\\preprocessed"

In [19]:
files = os.listdir(root)

for f in tqdm(files):
    text_file = open(os.path.join(root, f), 'r', encoding = 'utf-8')
    content = text_file.read()

    data = dict()
    data['text'] = preprocess_text(content)

    with open(os.path.join(dest, str(f[0:-3]) + "json"), 'w', encoding='utf-8') as outfile:
        json.dump(data, outfile)

 14%|█▍        | 422/3000 [16:27<1:40:32,  2.34s/it]


ValueError: [E088] Text of length 1282562 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.